## TASK 1

We want to create several models. It is going to be be a binary classification: good vs bad. Per each instrument. Per each famiy and with transfer learning from the family (maybe excluding trumpet) to the particular instrument. At the end I will create a detector of violin events.

| instrument | family | count\(sounds.id\) |
| :--- | :--- | :--- |
| bass | string | 159 |
| cello | string | 948 |
| clarinet | wind | 1688 |
| flute | wind | 980 |
| oboe | wind | 247 |
| piccolo | wind | 388 |
| sax\_alto | wind | 720 |
| sax\_baritone | wind | 288 |
| sax\_soprano | wind | 334 |
| sax\_tenor | wind | 680 |
| trumpet | wind | 934 |
| violin | string | 1383 |

We have to stratify by klass for not biasing.

In the M5 architecture, filters or convolutional layers are trained on the processed raw audio and it is not possible to transfer learning from very different sounds. Therefore, the strategy followed in this task is to use other instruments of the string family to train the sound quality assessment on the target. For example, training cello and violin and transfer learning to bass. For this purpose, M5 is trained for the rest of the string family. Then, the layers marked as a in Figure x are frozen and the classifier B is trained with the sounds of the target.



First of all we need to find out what version of cuda we have on the server. 

In [2]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


Then we install the pytorch version for the corresponding cuda version and the other packages.

In [3]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install pytorch-lightning==1.1
!pip uninstall -y torchtext
!pip install scikit-learn
!pip install git+https://github.com/mir-dataset-loaders/mirdata.git@Pedro/good_sounds

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1MB 1.2MB/s eta 0:04:22tcmalloc: large alloc 1147494400 bytes == 0x5649757c0000 @  0x7f3f53b8f615 0x56493bc0606c 0x56493bce5eba 0x56493bc08e8d 0x56493bcfa99d 0x56493bc7cfe9 0x56493bc77b0e 0x56493bc0a77a 0x56493bc7ce50 0x56493bc77b0e 0x56493bc0a77a 0x56493bc7986a 0x56493bcfb7c6 0x56493bc78ee2 0x56493bcfb7c6 0x56493bc78ee2 0x56493bcfb7c6 0x56493bc78ee2 0x56493bcfb7c6 0x56493bd7d431 0x56493bcde049 0x56493bc48c84 0x56493bc098e9 0x56493bc7dade 0x56493bc0a69a 0x56493bc78a45 0x56493bc77e0d 0x56493bc0a77a 0x56493bc78a45 0x56493bc0a69a 0x56493bc78a45
     |█████████████████████████████▏  | 1055.7MB 1.2MB/s eta 0:01:22tcmalloc: large alloc 1434370048 bytes == 0x5649b9e16000 @  0x7f3f53b8f615 0x56493bc0606c 0x56493bce5eba 0x56493bc08e8d 0x56493bcfa99d 0x56493bc7cfe9 0x56493bc77b0e 0x56493bc0a77a 0x56493bc7ce50 0x56493bc77b0e 0x56493bc0a77a 0x56493bc7986a 0x56493bcfb7c6 0x56493bc78ee2 0

We import the packages and we set the random_seed for our experiments. The random seed makes sure the experiment is reproducible on this environment.

We use mirdata to load the datasets, sklearn for data partitioning, torchaudio to load and transform audio files, and pytorch lightning on top of pytorch for machine learning. 

In [6]:
import mirdata
import numpy as np
import sklearn
import random
import torch
import torchaudio
import pytorch_lightning as pl
random_seed=0
pl.utilities.seed.seed_everything(seed=random_seed)

/usr/local/lib/python3.7/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


0

## Data processing

In [7]:
from google.colab import drive
drive.mount('/content/drive')

g = mirdata.initialize("good_sounds", data_home="drive/MyDrive/good_sounds")

Mounted at /content/drive


In [25]:
print(g.choice_track().get_sound_info)

{'id': 8427, 'instrument': 'piccolo', 'note': 'C', 'octave': 7, 'dynamics': None, 'recorded_at': '2016-01-28 10:00:00.000000', 'location': 'upf studio', 'player': 'irene', 'bow_velocity': None, 'bridge_position': None, 'string': None, 'csv_file': None, 'csv_id': '-10 render 023', 'pack_filename': '0222.wav', 'pack_id': 60, 'attack': None, 'decay': None, 'sustain': None, 'release': None, 'offset': None, 'reference': 1, 'klass': 'bad-timbre', 'comments': None, 'semitone': 84, 'pitch_reference': 442}



In order to use this dataset with pytorch, we extend the Dataset object to load the audio and annotations in our dataset, according to these [instructions](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html).

We basically need to write three methods:


*   __init__
*   __len__
*   __getitem__ to return each pair of audio array and class label


This is how a prototype of this class could look like:

```
class good_soundsDataset(torch.utils.data.Dataset):
    def __init__(self):
      self.track_ids = dataset.track_ids
    def __getitem__(self, index):
      # load data
      audio = load_audio(self.track_ids[index])
      label = self.track_ids[index].label
      # split audio in a fixed size array
      audio = audio[:seq_duration] 
      return audio,label
    def __len__(self):
      return len(self.tracks_ids)

```

Let's implement the class:



In [40]:

def get_data(dataset, klasses):
    # import pdb; pdb.set_trace()
    sound_selected = []
    track_ids, tracks = [], []
    if klasses == "all":
        for k, t in dataset.load_tracks().items():
            if t.get_sound_info['klass'] and t.get_sound_info['id'] not in sound_selected:
                track_ids.append(k)
                tracks.append(t)
                sound_selected.append(t.get_sound_info['id'])
    else:
        for klass in klasses:
          for k, t in dataset.load_tracks().items():
              if t.get_sound_info['instrument'] == klass and t.get_sound_info['klass'] and t.get_sound_info['id'] not in sound_selected:
                  track_ids.append(k)
                  tracks.append(t)
                  sound_selected.append(t.get_sound_info['id'])
    return track_ids, tracks


class good_soundsDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        mirdataset,
        seq_duration=0.5,
        random_start=True,
        resample=8000,
        subset=0,
        train_split=0.8,
        test_split=0.2,
        random_seed=42,
        klasses=["violin"]
    ):
        """
        """
        self.klasses = klasses # a 'klass' or 'all' or 'family'
        self.seq_duration = seq_duration
        self.dataset = mirdataset
        track_ids, tracks = get_data(self.dataset, self.klasses)
        self.track_ids = track_ids
        self.tracks = tracks
        self.resample = resample
        self.set = subset
        self.random_start = random_start

        #### build a list with labels
        self.labels = {label:i for i,label in enumerate(['good', 'bad'])}
        full_labels = [x.get_sound_info['klass'] for x in tracks]
        #### build the three subsets: train, validation, test using train_test_split, a stratified split with the labels
        self.trackids_train, self.trackids_test = sklearn.model_selection.train_test_split(self.track_ids, train_size=1-test_split, random_state=random_seed, stratify=full_labels)
        train_labels = [l for l,i in zip(full_labels, self.track_ids) if i in self.trackids_train]
        self.trackids_train, self.trackids_valid = sklearn.model_selection.train_test_split(self.trackids_train, train_size=train_split, random_state=random_seed, stratify=train_labels)


    def __getitem__(self, index):

        #### get the file with index in the corresponding subset
        if self.set==0:
            track_id = self.trackids_train[index]
        elif self.set==1:
            track_id = self.trackids_valid[index]
        elif self.set==2:
            track_id = self.trackids_test[index]
        track = self.dataset.track(track_id)

        #### compute start and end frames to read from the disk
        si, ei = torchaudio.info(track.audio_path)
        sample_rate, channels, length = si.rate, si.channels, si.length
        duration = length / sample_rate
        if self.seq_duration>duration:
            offset = 0
            num_frames = length
        else:
            if self.random_start:
                #### we skip a number of seconds at the beginning of file 
                start = random.uniform(0, duration - self.seq_duration)
            else:
                start = 0.
            #### seconds to audio frames
            offset = int(np.floor(start * sample_rate))
            num_frames = int(np.floor(self.seq_duration * sample_rate))


        #### get audio frames corresponding to offset and num_frames from the disk
        audio_signal, sample_rate = torchaudio.load(filepath=track.audio_path, offset=offset,num_frames=num_frames)

        #### zero pad if the size is smaller than seq_duration
        seq_duration_samples = int(self.seq_duration * sample_rate)
        total_samples = audio_signal.shape[-1]
        if seq_duration_samples>total_samples:
            audio_signal = torch.nn.ConstantPad2d((0,seq_duration_samples-total_samples,0,0),0)(audio_signal)

        #### resample
        audio_signal = torchaudio.transforms.Resample(sample_rate, self.resample)(audio_signal)

        return audio_signal, self.labels['good' if track.get_sound_info['klass'].startswith('good') else 'bad']

    def __len__(self):
        if self.set==0:
            return len(self.trackids_train)
        elif self.set==1:
            return len(self.trackids_valid)
        else:
            return len(self.trackids_test)



We initialize the dataset objects for train, validation, and test. We define the corresponding pytorch objects for data loading, defining the batch_size (paralellization on the GPU) and the num_workers ( data loading paralellization on CPU/memory).

In [41]:
#### Pytorch dataset loaders
train_dataset_v = good_soundsDataset(mirdataset=g, subset=0, random_seed=random_seed)
train_loader_v = torch.utils.data.DataLoader(train_dataset_v,batch_size=64,num_workers=4,pin_memory=True)
valid_dataset_v = good_soundsDataset(mirdataset=g, subset=1, random_seed=random_seed)
valid_loader_v = torch.utils.data.DataLoader(valid_dataset_v,batch_size=64,num_workers=4,pin_memory=True)
test_dataset_v = good_soundsDataset(mirdataset=g, subset=2, random_seed=random_seed)
test_loader_v = torch.utils.data.DataLoader(test_dataset_v,batch_size=64,num_workers=4,pin_memory=True)

In [42]:
len(train_dataset_v), len(valid_dataset_v), len(test_dataset_v)

(884, 222, 277)

In [43]:
string = {
    "v" : {
        "k": ["violin"]
    },
    "c" : {
        "k": ["cello"]
    },
    "b" : {
        "k": ["bass"]
    },
    "vc" : {
        "k": ["violin", "cello"]
    },
    "vb" : {
        "k": ["violin", "bass"]
    },
    "cb" : {
        "k": ["cello", "bass"]
    },
}
for k in string.keys():
    string[k]['train_dataset'] = good_soundsDataset(mirdataset=g, subset=0, random_seed=random_seed, klasses=string[k]['k'])
    string[k]['train_loader'] = torch.utils.data.DataLoader(string[k]['train_dataset'],batch_size=64,num_workers=4,pin_memory=True)
    string[k]['valid_dataset'] = good_soundsDataset(mirdataset=g, subset=1, random_seed=random_seed, klasses=string[k]['k'])
    string[k]['valid_loader'] = torch.utils.data.DataLoader(string[k]['valid_dataset'],batch_size=64,num_workers=4,pin_memory=True)
    string[k]['test_dataset'] = good_soundsDataset(mirdataset=g, subset=2, random_seed=random_seed, klasses=string[k]['k'])
    string[k]['test_loader'] = torch.utils.data.DataLoader(string[k]['test_dataset'],batch_size=64,num_workers=4,pin_memory=True)

In [44]:

for k in string.keys():
    print(k, len(string[k]['train_dataset']), len(string[k]['valid_dataset']), len(string[k]['test_dataset']))

v 884 222 277
c 464 116 146
b 101 26 32
vc 1349 338 422
vb 986 247 309
cb 566 142 177



**Which batch size/learning rate?**

Theory suggests that when multiplying the batch size by k, one should multiply the learning rate by sqrt(k) to keep the variance in the gradient expectation constant. See page 5 at A. Krizhevsky. One weird trick for parallelizing convolutional neural networks: https://arxiv.org/abs/1404.5997

However, recent experiments with large mini-batches suggest for a simpler linear scaling rule, i.e multiply your learning rate by k when using mini-batch size of kN. See P.Goyal et al.: Accurate, Large Minibatch SGD: Training ImageNet in 1 Hour https://arxiv.org/abs/1706.02677

## Training a pytorch lightning classifier

We extend the pytorch lightning module according to the [documentation](https://pytorch-lightning.readthedocs.io/en/stable/new-project.html). This may contain a definition of the layers in the neural network and how the data flows (how the layers are connected). You may overwrite other functions from `pl.LightningModule`, as described [here](https://pytorch-lightning.readthedocs.io/en/latest/lightning_module.html). The most important are `training_step` and `configure_optimizers`, in which we define the training loss and the optimizers. 

```
>>> class LitModel(pl.LightningModule):
...
...     def __init__(self):
...         super().__init__()
...         self.l1 = torch.nn.Linear(28 * 28, 10)
...
...     def forward(self, x):
...         return torch.relu(self.l1(x.view(x.size(0), -1)))
...
...     def training_step(self, batch, batch_idx):
...         x, y = batch
...         y_hat = self(x)
...         loss = F.cross_entropy(y_hat, y)
...         return loss
...
...     def configure_optimizers(self):
...         return torch.optim.Adam(self.parameters(), lr=0.02)
```



To predict the 2 classes  of the Gooud-sounds dataset on the raw audio files, we train a version of the M5 neural network which has been used in speech command recognition using waveforms. 

In [45]:
class M5_1(pl.LightningModule):
    '''
    M5 neural net taken from: https://pytorch.org/tutorials/intermediate/speech_command_recognition_with_torchaudio.html
    '''
    def __init__(self, n_input=1, n_output=10, stride=8, n_channel=32):
        super().__init__()
        #### network
        self.conv1 = torch.nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = torch.nn.BatchNorm1d(n_channel)
        self.pool1 = torch.nn.MaxPool1d(4)
        self.conv2 = torch.nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = torch.nn.BatchNorm1d(n_channel)
        self.pool2 = torch.nn.MaxPool1d(4)
        self.conv3 = torch.nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = torch.nn.BatchNorm1d(2 * n_channel)
        self.pool3 = torch.nn.MaxPool1d(4)
        self.conv4 = torch.nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = torch.nn.BatchNorm1d(2 * n_channel)
        self.pool4 = torch.nn.MaxPool1d(4)
        

        #### metrics
        self.train_acc = pl.metrics.Accuracy()
        self.valid_acc = pl.metrics.Accuracy()
        self.test_acc = pl.metrics.Accuracy()
        self.test_cm = pl.metrics.classification.ConfusionMatrix(num_classes=n_output)


    def forward(self, x):
        x = self.conv1(x)
        x = torch.nn.functional.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = torch.nn.functional.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = torch.nn.functional.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = torch.nn.functional.relu(self.bn4(x))
        x = self.pool4(x)
        # x = torch.nn.functional.avg_pool1d(x) #, kernel_size=x.shape[-1],stride=1
        x = x.permute(0, 2, 1)
        return x

    def training_step(self, batch, batch_idx):
        waveform, label = batch
        output = self.forward(waveform)
        ### why log softmax and nll loss: https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/
        loss = torch.nn.functional.nll_loss(output, label)
        self.log('train_loss', loss)
        self.train_acc(output, label)
        self.log('train_acc', self.train_acc, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        waveform, label = batch
        output = self.forward(waveform)
        loss = torch.nn.functional.nll_loss(output, label)
        self.log('val_loss', loss)
        self.valid_acc(output, label)
        self.log('valid_acc', self.valid_acc, on_step=True, on_epoch=True)

    def test_step(self, batch, batch_idx):
        waveform, label = batch
        output = self.forward(waveform)
        loss = torch.nn.functional.nll_loss(output, label)
        self.log('test_loss', loss)
        self.test_acc(output, label)
        self.log('test_acc', self.test_acc, on_step=True, on_epoch=True)
        self.test_cm(output, label)

    def training_epoch_end(self, outputs):
        # log epoch metric
        self.log('train_acc', self.train_acc.compute(), prog_bar=True)

    def validation_epoch_end(self, outputs):
        self.log('val_acc', self.valid_acc.compute(), prog_bar=True)

    def get_progress_bar_dict(self):
        # don't show the version number
        items = super().get_progress_bar_dict()
        items.pop("v_num", None)
        return items

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=2e-2,weight_decay=0.0001)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)  # reduce the learning after 10 epochs by a factor of 10
        return [optimizer], [scheduler]


class M5(pl.LightningModule):
    '''
    M5 neural net taken from: https://pytorch.org/tutorials/intermediate/speech_command_recognition_with_torchaudio.html
    '''
    def __init__(self, n_input=1, n_output=10, stride=8, n_channel=32, is_frozen=False, feature_extractor=None):
        super().__init__()
        #### network
        if not is_frozen:
            self.feature_extractor = M5_1(n_input, n_output, stride, n_channel)
        else:
            self.feature_extractor = feature_extractor
            self.feature_extractor.eval()
            self.feature_extractor.freeze()

        self.fc1 = torch.nn.Linear(2 * n_channel, n_output)

        #### metrics
        self.train_acc = pl.metrics.Accuracy()
        self.valid_acc = pl.metrics.Accuracy()
        self.test_acc = pl.metrics.Accuracy()
        self.test_cm = pl.metrics.classification.ConfusionMatrix(num_classes=n_output)


    def forward(self, x):
        x = self.feature_extractor.forward(x)
        x = self.fc1(x)
        return torch.nn.functional.log_softmax(x, dim=2).squeeze(1)

    def training_step(self, batch, batch_idx):
        waveform, label = batch
        output = self.forward(waveform)
        ### why log softmax and nll loss: https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/
        loss = torch.nn.functional.nll_loss(output, label)
        self.log('train_loss', loss)
        self.train_acc(output, label)
        self.log('train_acc', self.train_acc, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        waveform, label = batch
        output = self.forward(waveform)
        loss = torch.nn.functional.nll_loss(output, label)
        self.log('val_loss', loss)
        self.valid_acc(output, label)
        self.log('valid_acc', self.valid_acc, on_step=True, on_epoch=True)

    def test_step(self, batch, batch_idx):
        waveform, label = batch
        output = self.forward(waveform)
        loss = torch.nn.functional.nll_loss(output, label)
        self.log('test_loss', loss)
        self.test_acc(output, label)
        self.log('test_acc', self.test_acc, on_step=True, on_epoch=True)
        self.test_cm(output, label)

    def training_epoch_end(self, outputs):
        # log epoch metric
        self.log('train_acc', self.train_acc.compute(), prog_bar=True)

    def validation_epoch_end(self, outputs):
        self.log('val_acc', self.valid_acc.compute(), prog_bar=True)

    def get_progress_bar_dict(self):
        # don't show the version number
        items = super().get_progress_bar_dict()
        items.pop("v_num", None)
        return items

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=2e-2,weight_decay=0.0001)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)  # reduce the learning after 10 epochs by a factor of 10
        return [optimizer], [scheduler]


We train the model defined above.

In [49]:
#### Initialize the model
model = M5(n_input=train_dataset_v[0][0].shape[0], n_output=len(train_dataset_v.labels), is_frozen=False)

#### Initialize a trainer
trainer = pl.Trainer(gpus=1, max_epochs=20, progress_bar_refresh_rate=10)

#### Train the model
trainer.fit(model, train_loader_v, valid_loader_v)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
INFO: GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO: TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type            | Params
------------------------------------------------------
0 | feature_extractor | M5_1            | 24.6 K
1 | fc1               | Linear          | 130   
2 | train_acc         | Accuracy        | 0     
3 | valid_acc         | Accuracy        | 0     
4 | test_acc          | Accuracy        | 0     
5 | test_cm           | ConfusionMatrix | 0     
------------------------------------------------------
24.8 K    Trainable params
0    

1

Once the model is trained we can use it to process data, save it, get the metrics on the test set. 

In [52]:
#### Put the model in production
model.eval()

#### Compute metrics on the test set
trainer.test(test_dataloaders=test_loader_v)




--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': Accuracy(),
 'test_acc_epoch': tensor(0.8448, device='cuda:0'),
 'test_loss': tensor(0.4558, device='cuda:0'),
 'val_acc': tensor(0.8108, device='cuda:0'),
 'val_loss': tensor(0.6007, device='cuda:0'),
 'valid_acc': Accuracy(),
 'valid_acc_epoch': tensor(0.8108, device='cuda:0')}
--------------------------------------------------------------------------------


[{'test_acc': Accuracy(),
  'test_acc_epoch': 0.8447653651237488,
  'test_loss': 0.45580288767814636,
  'val_acc': 0.8108108043670654,
  'val_loss': 0.6007232666015625,
  'valid_acc': Accuracy(),
  'valid_acc_epoch': 0.8108108043670654}]

## transfer from other string instruments to violin

In [65]:

def deploy_model(loader, is_transfer, transfer_model=None):
    #### Initialize the model
    if is_transfer:
        model = M5(n_input=loader['train_dataset'][0][0].shape[0], n_output=len(loader['train_dataset'].labels), is_frozen=True, feature_extractor=transfer_model)
    else:
        model = M5(n_input=loader['train_dataset'][0][0].shape[0], n_output=len(loader['train_dataset'].labels), is_frozen=False)
    #### Initialize a trainer
    trainer = pl.Trainer(gpus=1, max_epochs=20, progress_bar_refresh_rate=10)

    #### Train the model
    trainer.fit(model, loader['train_loader'], loader['valid_loader'])

    #### Put the model in production
    model.eval()

    #### Compute metrics on the test set
    trainer.test(test_dataloaders=loader['test_loader'])
    return model
    

violin_model = deploy_model(string['v'], False)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
INFO: GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO: TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type            | Params
------------------------------------------------------
0 | feature_extractor | M5_1            | 24.6 K
1 | fc1               | Linear          | 130   
2 | train_acc         | Accuracy        | 0     
3 | valid_acc         | Accuracy        | 0     
4 | test_acc          | Accuracy        | 0     
5 | test_cm           | ConfusionMatrix | 0     
------------------------------------------------------
24.8 K    Trainable params
0    


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': Accuracy(),
 'test_acc_epoch': tensor(0.8339, device='cuda:0'),
 'test_loss': tensor(0.4424, device='cuda:0'),
 'val_acc': tensor(0.8153, device='cuda:0'),
 'val_loss': tensor(0.5048, device='cuda:0'),
 'valid_acc': Accuracy(),
 'valid_acc_epoch': tensor(0.8153, device='cuda:0')}
--------------------------------------------------------------------------------


In [66]:
cello_model = deploy_model(string['c'], False)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
INFO: GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO: TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type            | Params
------------------------------------------------------
0 | feature_extractor | M5_1            | 24.6 K
1 | fc1               | Linear          | 130   
2 | train_acc         | Accuracy        | 0     
3 | valid_acc         | Accuracy        | 0     
4 | test_acc          | Accuracy        | 0     
5 | test_cm           | ConfusionMatrix | 0     
------------------------------------------------------
24.8 K    Trainable params
0    


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': Accuracy(),
 'test_acc_epoch': tensor(0.8151, device='cuda:0'),
 'test_loss': tensor(0.4848, device='cuda:0'),
 'val_acc': tensor(0.8534, device='cuda:0'),
 'val_loss': tensor(0.4159, device='cuda:0'),
 'valid_acc': Accuracy(),
 'valid_acc_epoch': tensor(0.8534, device='cuda:0')}
--------------------------------------------------------------------------------


In [67]:
bass_model = deploy_model(string['b'], False)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
INFO: GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO: TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type            | Params
------------------------------------------------------
0 | feature_extractor | M5_1            | 24.6 K
1 | fc1               | Linear          | 130   
2 | train_acc         | Accuracy        | 0     
3 | valid_acc         | Accuracy        | 0     
4 | test_acc          | Accuracy        | 0     
5 | test_cm           | ConfusionMatrix | 0     
------------------------------------------------------
24.8 K    Trainable params
0    


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': Accuracy(),
 'test_acc_epoch': tensor(0.8125, device='cuda:0'),
 'test_loss': tensor(0.3745, device='cuda:0'),
 'val_acc': tensor(0.8846, device='cuda:0'),
 'val_loss': tensor(0.2994, device='cuda:0'),
 'valid_acc': Accuracy(),
 'valid_acc_epoch': tensor(0.8846, device='cuda:0')}
--------------------------------------------------------------------------------


In [68]:
vc_model = deploy_model(string['vc'], False)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
INFO: GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO: TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type            | Params
------------------------------------------------------
0 | feature_extractor | M5_1            | 24.6 K
1 | fc1               | Linear          | 130   
2 | train_acc         | Accuracy        | 0     
3 | valid_acc         | Accuracy        | 0     
4 | test_acc          | Accuracy        | 0     
5 | test_cm           | ConfusionMatrix | 0     
------------------------------------------------------
24.8 K    Trainable params
0    


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': Accuracy(),
 'test_acc_epoch': tensor(0.8365, device='cuda:0'),
 'test_loss': tensor(0.4713, device='cuda:0'),
 'val_acc': tensor(0.8432, device='cuda:0'),
 'val_loss': tensor(0.4257, device='cuda:0'),
 'valid_acc': Accuracy(),
 'valid_acc_epoch': tensor(0.8432, device='cuda:0')}
--------------------------------------------------------------------------------


In [69]:
vb_model = deploy_model(string['vb'], False)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
INFO: GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO: TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type            | Params
------------------------------------------------------
0 | feature_extractor | M5_1            | 24.6 K
1 | fc1               | Linear          | 130   
2 | train_acc         | Accuracy        | 0     
3 | valid_acc         | Accuracy        | 0     
4 | test_acc          | Accuracy        | 0     
5 | test_cm           | ConfusionMatrix | 0     
------------------------------------------------------
24.8 K    Trainable params
0    


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': Accuracy(),
 'test_acc_epoch': tensor(0.8220, device='cuda:0'),
 'test_loss': tensor(0.4560, device='cuda:0'),
 'val_acc': tensor(0.8138, device='cuda:0'),
 'val_loss': tensor(0.6642, device='cuda:0'),
 'valid_acc': Accuracy(),
 'valid_acc_epoch': tensor(0.8138, device='cuda:0')}
--------------------------------------------------------------------------------


In [70]:
cb_model = deploy_model(string['cb'], False)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
INFO: GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO: TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type            | Params
------------------------------------------------------
0 | feature_extractor | M5_1            | 24.6 K
1 | fc1               | Linear          | 130   
2 | train_acc         | Accuracy        | 0     
3 | valid_acc         | Accuracy        | 0     
4 | test_acc          | Accuracy        | 0     
5 | test_cm           | ConfusionMatrix | 0     
------------------------------------------------------
24.8 K    Trainable params
0    


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': Accuracy(),
 'test_acc_epoch': tensor(0.7514, device='cuda:0'),
 'test_loss': tensor(0.5438, device='cuda:0'),
 'val_acc': tensor(0.8169, device='cuda:0'),
 'val_loss': tensor(0.4849, device='cuda:0'),
 'valid_acc': Accuracy(),
 'valid_acc_epoch': tensor(0.8169, device='cuda:0')}
--------------------------------------------------------------------------------


In [71]:
bass_transfer = deploy_model(string['b'], True, transfer_model=vc_model.feature_extractor)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
INFO: GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO: TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type            | Params
------------------------------------------------------
0 | feature_extractor | M5_1            | 24.6 K
1 | fc1               | Linear          | 130   
2 | train_acc         | Accuracy        | 0     
3 | valid_acc         | Accuracy        | 0     
4 | test_acc          | Accuracy        | 0     
5 | test_cm           | ConfusionMatrix | 0     
------------------------------------------------------
130       Trainable params
24.6 


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': Accuracy(),
 'test_acc_epoch': tensor(0.6562, device='cuda:0'),
 'test_loss': tensor(0.6270, device='cuda:0'),
 'val_acc': tensor(0.5000, device='cuda:0'),
 'val_loss': tensor(0.6957, device='cuda:0'),
 'valid_acc': Accuracy(),
 'valid_acc_epoch': tensor(0.5000, device='cuda:0')}
--------------------------------------------------------------------------------


In [72]:
cello_transfer = deploy_model(string['c'], True, transfer_model=vb_model.feature_extractor)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
INFO: GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO: TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type            | Params
------------------------------------------------------
0 | feature_extractor | M5_1            | 24.6 K
1 | fc1               | Linear          | 130   
2 | train_acc         | Accuracy        | 0     
3 | valid_acc         | Accuracy        | 0     
4 | test_acc          | Accuracy        | 0     
5 | test_cm           | ConfusionMatrix | 0     
------------------------------------------------------
130       Trainable params
24.6 


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': Accuracy(),
 'test_acc_epoch': tensor(0.8151, device='cuda:0'),
 'test_loss': tensor(0.5016, device='cuda:0'),
 'val_acc': tensor(0.8534, device='cuda:0'),
 'val_loss': tensor(0.4232, device='cuda:0'),
 'valid_acc': Accuracy(),
 'valid_acc_epoch': tensor(0.8534, device='cuda:0')}
--------------------------------------------------------------------------------


In [73]:
violin_transfer = deploy_model(string['v'], True, transfer_model=cb_model.feature_extractor)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
INFO: GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO: TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type            | Params
------------------------------------------------------
0 | feature_extractor | M5_1            | 24.6 K
1 | fc1               | Linear          | 130   
2 | train_acc         | Accuracy        | 0     
3 | valid_acc         | Accuracy        | 0     
4 | test_acc          | Accuracy        | 0     
5 | test_cm           | ConfusionMatrix | 0     
------------------------------------------------------
130       Trainable params
24.6 


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': Accuracy(),
 'test_acc_epoch': tensor(0.8448, device='cuda:0'),
 'test_loss': tensor(0.4317, device='cuda:0'),
 'val_acc': tensor(0.8333, device='cuda:0'),
 'val_loss': tensor(0.4513, device='cuda:0'),
 'valid_acc': Accuracy(),
 'valid_acc_epoch': tensor(0.8333, device='cuda:0')}
--------------------------------------------------------------------------------
